In [33]:
import sys
sys.path.append('../')
import config
import tweepy
import os
import json
import codecs

import pandas as pd
import datetime
from datetime import datetime, timedelta
import locale

In [34]:
# 認証部分を関数にして、clientを返す
def auth():
    # API KEY
    CK = config.CK
    CS = config.CS
    AT = config.AT
    AS = config.AS
    BT = config.BT

    # tweepy setting
    # auth = tweepy.OAuthHandler(CK, CS)
    # auth.set_access_token(AT, AS)
    

    # wait_on_rate_limit = True とするとレートリミットを確認しながら取得処理を行う
    client = tweepy.Client(bearer_token = BT,\
                           consumer_key = CK,\
                           consumer_secret = CS,\
                           access_token = AT,\
                           access_token_secret = AS,\
                           wait_on_rate_limit = True)

    return client

In [35]:
# Use tweepy.Client()
client = auth()

In [36]:
# Define a search_term for searching tweets
search_term = "CVE"

In [38]:
# get recent 20 tweets
tweets = client.search_recent_tweets(query = search_term,\
                                     max_results = 20\
                                    )

In [39]:
print(type(tweets))

<class 'tweepy.client.Response'>


In [42]:
if tweets is not None:
    for tweet in tweets[0]:
        print(tweet)

RT @threatintelctr: 🚨 NEW: CVE-2018-1065 🚨 The netfilter subsystem in the Linux kernel through 4.15.7 mishandles the case of a rule blob th…
RT @sachin_pandey98: How I got $$$$ Bounty within 5 mins for CVE-2017–9248 by @7H3P4n7h3R https://t.co/3pJw9RobBq
CVE-2023-0590 | Linux Kernel net/sched/sch_api.c qdisc_graft use after free A vulnerability was found in Linux Kernel. It has been declared as problematic. This vulnerability affects the function qdisc_graft of the file net/sched/sch_api.c. The manipulat… https://t.co/XVz6Ljwu38
CVE-2023-22367 | Ichiran App up to 3.0.x on Android/iOS certificate validation A vulnerability was found in Ichiran App up to 3.0.x. It has been classified as problematic. This affects an unknown part. The manipulation leads to improper certificate valid… https://t.co/BoxNlKq2qB
CVE-2023-22362 | SUSHIRO App on Android log file A vulnerability was found in SUSHIRO App and classified as problematic. Affected by this issue is some unknown functionality. The manipu

In [18]:
# 認証部分を関数にして、apiを返す
def create_api():
    # API KEY
    CK = config.CK
    CS = config.CS
    AT = config.AT
    AS = config.AS

    # tweepy setting
    auth = tweepy.OAuthHandler(CK, CS)
    auth.set_access_token(AT, AS)

    # wait_on_rate_limit = True とするとレートリミットを確認しながら取得処理を行う
    api = tweepy.API(auth, wait_on_rate_limit = True)

    return api

In [31]:
#処理を開始した日時を取得する
start = datetime.datetime.today()
startdate=start.strftime("%Y%m%d%H%M%S")

#リスト作成
list_id_str = []
list_screen_name = []
list_username = []
list_userid = []
list_created_at = []
list_text = []

In [105]:
# https://qiita.com/daisuke-aoki/items/30e3d6e84637326ef1da
# 取得したいツイート数
limit = 50000            

# 取得対象のツイートの時間幅を指定する この例では実行前の２４時間
# iso形式のUTC時間で指定しないと正しく時間指定ができない模様。
# 指定した時間幅に、limitで指定した件数以上のツイートがあってもlimit以上は取得しない

now = datetime.now()
print(now)
start_time = now - timedelta(days=6)
start_time = start_time.replace(hour = 0, minute = 0, second=0, microsecond=0)
print(start_time)
end_time = now.replace(hour = 0, minute = 0, second=0, microsecond=0)
print(end_time)
# now = now.replace(minute=0, second=0, microsecond=0)
end_time_tweepy = str(end_time.isoformat()) +'+09:00'
# start_time = now - timedelta(days=7) 
start_time_tweepy = str(start_time.isoformat())+'+09:00'
print(start_time_tweepy)
print(end_time_tweepy)

2023-02-13 11:43:16.576635
2023-02-07 00:00:00
2023-02-13 00:00:00
2023-02-07T00:00:00+09:00
2023-02-13T00:00:00+09:00


In [109]:
# f = codecs.open('tweets[{}]_{}.txt'.format(search_term,startdate), 'w', 'utf-8')
client = auth()
#itr = tweepy.Cursor(api.followers_ids, screen_name = screen_name, count=5000,cursor=cursor).pages()
# itr = tweepy.Cursor(api.search_tweets, q = search_term, tweet_mode="extended", count = 200, cursor=cursor).pages()
# print(type(itr))
# try:
#     for tweet in itr.next():
#         # print(tweet._json)
#         try:
#             # user = api.get_user(follower_id)
#             # tweet_info = [tweet.id_str, tweet.screen_name, tweet.name, tweet.created_at, tweet.full_text]
#             tweet_info = [tweet._json['id_str'], tweet._json['user']['screen_name'], tweet._json['user']['name'], tweet._json['user']['id'], tweet._json['created_at'], tweet._json['full_text']]
#             list_id_str.append(tweet._json['id_str'])
#             list_screen_name.append(tweet._json['user']['screen_name'])
#             list_username.append(tweet._json['user']['name'])
#             list_userid.append(tweet._json['user']['id'])
#             list_created_at.append(tweet._json['created_at'])
#             list_text.append(tweet._json['full_text'])
#             print(tweet_info)
#             f.write(str(tweet_info))
#             f.write("\n")
#         except tweepy.error.TweepError as e:
#             print("エラーあり")
#             print(e.reason)
#             f.write(e.reason + "\n")
# except ConnectionError as e:
#     print(e.reason)
#     f.write(e.reason + "\n")
# f.close()

#データフレーム作成
# df=pd.DataFrame({'id_str':list_id_str,'screen_name':list_screen_name,'username':list_username,'created_at':list_created_at, 'text':list_text})
df_tweet = pd.DataFrame()
for tweet in tweepy.Paginator(client.search_recent_tweets,\
                              query = search_term,\
                              start_time=start_time_tweepy,\
                              end_time=end_time_tweepy,\
                              tweet_fields=['id','created_at','text','author_id','public_metrics',],\
                              # expansions='author_id',\
                              # user_fields='description',\
                              #user_fields=['id','name','username',],\
                              max_results = 100).flatten(limit = limit):
    # print(dir(tweet))
    df_tweet = pd.concat([df_tweet, pd.DataFrame([tweet.data])])
    #df_tweet = pd.concat([df_tweet, pd.DataFrame([tweet.data], ignore_index=True)])

print(df_tweet)

#本日の日時を取得してファイル名の一部に設定
# d = datetime.datetime.today()
# filename=d.strftime("%Y%m%d%H%M%S")

# df.to_csv(filename+"_" + search_term + ".csv", index=False,header=False)
# f = codecs.open('tweets[{}]_{}.txt'.format(search_term,startdate), 'w', 'utf-8')
# print("Start " + startdate)
# print("End " + filename + ": 取得件数は"+str(len(df))+"件です")
# f.write("Start " + startdate)
# f.write("\n")
# f.write("End " + filename + ": 取得件数は"+str(len(df))+"件です")
# f.write("\n")
# f.close()

                    id                created_at            author_id  \
0  1624785106333310984  2023-02-12T14:58:51.000Z  1530877110226665472   
0  1624784970509348866  2023-02-12T14:58:18.000Z  1388217926541447169   
0  1624784895871680512  2023-02-12T14:58:00.000Z  1388217926541447169   
0  1624784592413532160  2023-02-12T14:56:48.000Z            941026441   
0  1624784417171419140  2023-02-12T14:56:06.000Z  1322315722287099909   
0  1624784406987567104  2023-02-12T14:56:04.000Z  1322315722287099909   
0  1624784230352924677  2023-02-12T14:55:22.000Z  1464325401224912901   
0  1624784198950158337  2023-02-12T14:55:14.000Z  1474773760968384521   
0  1624783987607764993  2023-02-12T14:54:24.000Z   861783161801326592   
0  1624783395438972932  2023-02-12T14:52:03.000Z  1425939384373764102   

                                      public_metrics  \
0  {'retweet_count': 0, 'reply_count': 1, 'like_c...   
0  {'retweet_count': 39, 'reply_count': 0, 'like_...   
0  {'retweet_count': 57, 're

In [79]:
filename = start_time.strftime("%Y%m%d") + "-" + end_time.strftime("%Y%m%d") 
print(filename)

20230207-20230213


In [80]:
df_tweet.to_csv(filename + ".csv", index=False,header=False)